Imports. The output of `tf.__version__` should read something like 2.#

In [1]:
import numpy as np
import os
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tflite_model_maker'

Count local devices. Make sure that a `GPU` is available here.

In [3]:
from tensorflow.python.client import device_lib
for d in device_lib.list_local_devices():
    print(d)
    print('---')

name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15523577509397833168

---
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7769554944
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9444538946916232917
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1"

---


2021-08-26 14:23:39.842208: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-26 14:23:40.107104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-08-26 14:23:40.614546: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 7409 MB memory:  -> device: 0, name: GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Set directories.

In [4]:
CLASS_LABELS = "./data/100k/classlabels.txt"
DATA_DIR = "./data/100k/"
CKPT_DIR = "./ckpts/"
MODEL_EXPORT_DIR = "./export/100kD2"
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(MODEL_EXPORT_DIR, exist_ok=True)

Read class labels.

In [5]:
with open(CLASS_LABELS, "r") as f:
    label_map = [s.strip() for s in f.readlines()]
    
label_map_dict = {}
for i, l in enumerate(label_map):
    label_map_dict[i+1] = l

for k, v in label_map_dict.items():
    print(k, ": ", v)

1 :  circle
2 :  diamond
3 :  halfcircle
4 :  heart
5 :  hexagon
6 :  hexstar
7 :  moon
8 :  octagon
9 :  pentagon
10 :  pentstar
11 :  plus
12 :  square
13 :  trapezoid
14 :  triangle


In [6]:
train_data = object_detector.DataLoader(
    tfrecord_file_patten = DATA_DIR + "100k_train.tfrecord",
    size = 90000,
    label_map = label_map_dict
)

test_data = object_detector.DataLoader(
    tfrecord_file_patten = DATA_DIR + "100k_test.tfrecord",
    size = 10000,
    label_map = label_map_dict
)

Create spec with hyperparameters.

In [7]:
spec = object_detector.EfficientDetLite2Spec(
    model_dir = CKPT_DIR,
    hparams = {
        "map_freq" : 2,
    },
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
)

2021-08-26 14:24:07.295707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-08-26 14:24:07.303633: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7409 MB memory:  -> device: 0, name: GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Train model

In [ ]:
model = object_detector.create(
    train_data = train_data,
    model_spec = spec,
    validation_data = test_data,
    epochs = 200,
    batch_size = 64,
    train_whole_model = False,
    do_train = True,
)

INFO:tensorflow:Retraining the models...


2021-08-26 14:24:41.115032: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200


2021-08-26 14:25:01.014121: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202


 647/1406 [============>.................] - ETA: 14:47 - det_loss: 1.0397 - cls_loss: 0.8232 - box_loss: 0.0043 - reg_l2_loss: 0.0072 - loss: 1.0469 - learning_rate: 0.0245 - gradient_norm: 0.6608